In [2]:
import nosbench
from nosbench.program import Program, Instruction, Pointer
from nosbench.function import Function, interpolate, bias_correct, clip, size
import torch

import pprint

from neps.space.new_space import space
from neps.space.new_space import config_string

EPOCHS_PER_CONFIG = 1
AVAILABLE_VARIABLE_SLOTS = 11 # As per the NOSBench paper

In [3]:


def _function_writer(operation: callable, *args, store: int) -> tuple[Function, list[Pointer], Pointer]:
    return Function(operation, len(args)), [Pointer(var) if isinstance(var, int) else Pointer(var()) for var in args], store
    
def program_compiler(instruction: tuple[callable, list[Pointer], Pointer], *args) -> Program:
    if len(args) == 0:
        return Program([Instruction(instruction[0], inputs=instruction[1], output=instruction[2])])
    else:
        return Program(program_compiler(instruction) + sum(args,[]))

class nosBench(space.Pipeline):
    _UNARY_FUN = space.Categorical(
        choices=(torch.square, torch.exp, torch.log, torch.sign, torch.sqrt, torch.abs, torch.norm, torch.sin, torch.cos, torch.tan, torch.asin, torch.acos, torch.atan, torch.mean, torch.std, size,)
    )

    _BINARY_FUN = space.Categorical(
        choices=(clip, torch.div, torch.mul, torch.add, torch.sub, torch.minimum, torch.maximum, torch.heaviside)
    )

    _TERNARY_FUN = space.Categorical(
        choices=(interpolate, bias_correct,)
    )

    _CONST = space.Integer(3, 8) # 0-2 are reserved for weights, gradient, and optimizer step
    _VAR = space.Integer(9, AVAILABLE_VARIABLE_SLOTS+9) # And 3-8 are reserved for the constants

    # Is this the right way to handle constants?
    _PARAMS = space.Categorical(
        choices=(
            0, # weights
            1, # gradient
            2, # optimizer step
        )
    )

    _POINTER = space.Categorical(
        choices=(
            space.Resampled(_PARAMS),
            space.Resampled(_CONST),
            space.Resampled(_VAR),
        ),
    )

    _F_ARGS = space.Categorical(
        choices=(
            (
                space.Resampled(_UNARY_FUN),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_BINARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_TERNARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
        ),
    )

    _F = space.Operation(
        operator=_function_writer,
        args=space.Resampled(_F_ARGS),
        kwargs={"store": space.Resampled(_VAR)}, # move to _F_ARGS as normal arg?
    )

    _P_ARGS = space.Categorical(
        choices=(
            (space.Resampled(_F),),
            (space.Resampled(_F), space.Resampled("P"),), # Can be multiple space.Resampled("P")
        ),
    )

    P = space.Operation(
        operator=program_compiler,
        args=space.Resampled(_P_ARGS),
    )


In [7]:
pipeline = nosBench()

resolved_pipeline, resolution_context = space.resolve(pipeline)

p = resolved_pipeline.P
p_config_string = space.convert_operation_to_string(p)
assert p_config_string 
pretty_config = config_string.ConfigString(p_config_string).pretty_format()
assert pretty_config

# print()
# print("Config string:")
# print(pretty_config)

program = space.convert_operation_to_callable(p)
pprint.pprint(program)

benchmark = nosbench.ToyBenchmark()
fitness = benchmark.query(program, EPOCHS_PER_CONFIG)
print(fitness)


[Instruction(Function(maximum, 2), inputs=[17, 6], output=16),
 Instruction(Function(mul, 2), inputs=[20, 10], output=15)]
1.1031578063964844


In [5]:
import time

t0 = time.perf_counter()
samples = 10_000

for _ in range(samples):
    resolved_pipeline, resolution_context = space.resolve(pipeline)

t1 = time.perf_counter()

print(f"sampling takes {(t1 - t0) / samples:.2}s on average over {samples} samples")
print(f"duration for {samples} samples: {t1 - t0:.3}s ")

KeyboardInterrupt: 